In [19]:
import os
from langchain_community.embeddings import OllamaEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import UnstructuredEPubLoader

from langchain_community.document_loaders import PyPDFLoader

embeddings = OllamaEmbeddings(model='nomic-embed-text:v1.5')

import logging

# setar logging como INFO
logging.basicConfig(level=logging.INFO)

In [20]:
path_books = './books/import'
path_done_books = './books/done'
path_vector_store = '../vector-store/books'

In [21]:
def load_books(path_books):
    logging.info(f'Loading books from {path_books}')
    books = []
    for root, dirs, files in os.walk(path_books):
        for file in files:
            if file.endswith('.pdf'):
                books.append(os.path.join(root, file))
    return books

def load_pdf(file):
    logging.info(f'Loading PDF {file}')
    loader = PyPDFLoader(file)
    documents = loader.load()
    return documents

def load_chunked_docs(documents):
    logging.info(f'Chunking documents')
    text_spliter = RecursiveCharacterTextSplitter(
        chunk_size=2500,
        chunk_overlap=100
    )
    chunked_docs = text_spliter.split_documents(documents)
    return chunked_docs

def load_vector_store(chunked_docs, path_vector_store):
    if not os.path.isdir(path_vector_store):
        logging.info(f'Creating vector store')
        os.makedirs(path_vector_store)
        db = FAISS.from_documents(chunked_docs, embeddings)
        db.save_local(path_vector_store)
        logging.info(f'Vector store loaded {db.index.ntotal}')
        return db
    else:
        logging.info(f'Loading vector store')
        db = FAISS.load_local(folder_path=path_vector_store, 
                              embeddings=embeddings, 
                              allow_dangerous_deserialization=True)
        logging.info(f'Adding documents to vector store')
        db.add_documents(chunked_docs)
        db.save_local(path_vector_store)
        logging.info(f'Vector store loaded {db.index.ntotal}')
        return db

In [22]:
books = load_books(path_books)
for book in books:
    logging.info(f'Processing book {book}')
    documents_books = load_pdf(book)
    chunked_docs = load_chunked_docs(documents_books)
    db = load_vector_store(chunked_docs=chunked_docs, path_vector_store=path_vector_store)
    logging.info(f'Moving book {book} to {path_done_books}')
    os.rename(book, os.path.join(path_done_books, os.path.basename(book)))

INFO:root:Loading books from ./books/import
INFO:root:Processing book ./books/import/Conselhos para obreiros - Charles H Spurgeon.pdf
INFO:root:Loading PDF ./books/import/Conselhos para obreiros - Charles H Spurgeon.pdf
INFO:root:Chunking documents
INFO:root:Creating vector store
INFO:root:Vector store loaded 145
INFO:root:Moving book ./books/import/Conselhos para obreiros - Charles H Spurgeon.pdf to ./books/done


In [17]:
#testar se o documento foi adicionado
#db = FAISS.load_local("../vector-store/books", embeddings, allow_dangerous_deserialization=True)
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 3

results = retriever.get_relevant_documents("Calvinismo")


In [18]:
for row in results:
    page_content = row.page_content
    source = row.metadata['source']
    page = row.metadata['page']

    

    # Agora você pode fazer o que quiser com esses dados.
    # Por exemplo, você pode imprimi-los:
    print(f"""
Page Content: 
{page_content.replace('	',' ')}

Source: 
{source} 

Page:
{page}

""")


Page Content: 
batalha confiando em nossa força, não devemos nos surpreender se voltarmos manchados pela
derrota. Se não fosse pelo teu poder, ó Espírito de Deus, não poderíamos nem tentar; mas,
quando confiamos em ti, seguimos adiante com fé.
Ultimamente tenho sido tocado, ao olhar a história da Reforma e das épocas anteriores ela, pela
franqueza do testemunho dos pregadores antigos. Se observarmos a vida de Farel, veremos que
ele não pregava sobre o evangelho, mas o evangelho. João Calvino fazia o mesmo. Hoje, é claro,
ele  é  visto  apenas  como  um  teólogo,  mas  na  verdade  foi  um  dos  maiores  pregadores  do
evangelho. Quando João Calvino abria o Livro e escolhia um texto, podem ter certeza de que ele
pregava: “Porque pela graça sois salvos, por meio da fé, e isto não vem de vós, é dom de Deus”
(Ef 2.8). O mesmo acontecia com Lutero. A pregação de Lutero era como um grande sino,
soando sempre a mesma nota: “Crê no Senhor Jesus Cristo e viverás! Não somos salvos por

Source: 

In [ ]:
#loader = UnstructuredEPubLoader("/home/cristian/Documentos/Projetos/personal-search/ingestion/livros/Livro Conselhos para obreiros - Charles Spurgeon.epub", mode="elements")
#data = loader.load()

#documents = loader.load()